In [1]:
import sys
import requests
import re
import bs4
from bs4 import BeautifulSoup
import urllib.request
import csv
from datetime import datetime
import pandas as pd
import numpy as np
import os


#获取HTML文本
def getHTMLText(url):
	try:
		#模拟浏览器
		kv = {'user-agent':'Mozilla/5.0'}
		r = requests.get(url, headers=kv, timeout=30)  
		r.raise_for_status()
		r.encoding = r.apparent_encoding
		return r.text
	except:
		print("InternetError!")
		return " "

In [2]:
#解析并且返回HTML文本
def HTMLTextconvert(html):
	try:
		soup = bs4.BeautifulSoup(html, "html.parser")
		return soup
	except:
		print("HTMLConvertError!")
		return " "
 

In [3]:
#检索HTML中的信息，获取搜索排名信息
#存在置顶的情况，需要特殊判断
def HTMLSearch(html, ranklist):
	links = []
	try:
		flag = 0
		 #找到所有tbody标签下的所有内容，并且遍历所有的儿子节点
		for tr in html.find("tbody").children:
			if isinstance(tr, bs4.element.Tag):
				if flag==0:
					rank = "置顶"
					#注意由于class属性会和python中的关键字重名，因此需要变为class_
					td02 = tr.find_all(class_=re.compile('td-02'))
					for i in td02:
							if isinstance(i, bs4.element.Tag):
									#trans得到的类型为列表
									trans = i.find_all("a")
									trans2 = i.find('a', href=True)['href']
					number = " "
					ranklist.append([rank, trans[0].string, number])
					links.append(trans2)
					#writer.writerow(trans[0])
					flag = 1
				else:
					#排名信息在td标签下的class=td-01属性中
					td01 = tr.find_all(class_=re.compile("td-01"))
					for i in td01:
							if isinstance(i, bs4.element.Tag):
									rank = i.string
					#热搜内容和搜索量在td标签下的class=td-02属性中：内容是a标签，搜索量是span标签
					td02 = tr.find_all(class_=re.compile("td-02"))
					for i in td02:
							name = i.find_all("a")
							column = i.find_all("span")
							name2 = i.find('a', href=True)['href']
					ranklist.append([rank, name[0].text, column[0].text])
					#writer.writerow(name[0])
					links.append(name2)
					#contents(name2)
		#return links
	except:
		print("HTMLSearchError!")
	contents(links,ranklist)

In [4]:
def saveFile(df):
    csv_file = "/Users/boli/Desktop/export.csv"

    if not os.path.isfile(csv_file):
        df.to_csv(csv_file, header=True, index=False, encoding = 'utf-8')
        print("new")
    else:
        df.to_csv(csv_file, mode='a', header=False, index=False)
        print("exist")

In [5]:
def saveFrame(data, topics, time, data_len):
    col_name = ["Date", "Topic", "Comment"]
    df = pd.DataFrame(columns = col_name)

    time_lst = []
    topic_lst = []
    
    for i in range(data_len):
        time_lst.append(time)
        topic_lst.append(topics)

    df["Date"] = time_lst
    df["Topic"] = topic_lst
    df["Comment"] = data

    saveFile(df)

In [6]:
def contents(links,ranklist): 
    time = datetime.now().strftime("%m/%d/%Y-%H:%M:%S") # time object

    # Retrieve links
    for i in range(1,35):  #从 1 开始
        topics = []
        data = []

        link = 'https://s.weibo.com'+links[i]
        url = urllib.request.urlopen(link).read()
        soup = bs4.BeautifulSoup(url,'lxml')

        all_contents = soup.findAll("div", {"class": "content"})

        for content in all_contents:
            if content.find("p", attrs={'node-type': 'feed_list_content_full', 'class': 'txt'}):
                expended_content = content.find("p", attrs={'node-type': 'feed_list_content_full', 'class': 'txt'}).get_text().replace("收起全文d", "")
                data.append(expended_content)
            else:
                data.append(content.find("p", {"class": "txt"}).get_text())

        if ranklist[i][1] not in topics:
            topics.append(ranklist[i][1])

        saveFrame(data, topics, new_period, len(data))  



03/31/2020, 00:00:00


In [7]:
def main():
    try:
        #微博热搜的网站
        url = "https://s.weibo.com/top/summary?Refer=top_hot&topnav=1&wvr=6"

        rank = []
        text = getHTMLText(url)
        soup = HTMLTextconvert(text)
        HTMLSearch(soup, rank)

    except:
        print("SystemError!")
        return 0

main()
print("Done")

exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
exist
Done


In [8]:
# Convert csv to DF
import pandas as pd

data = pd.read_csv("/Users/boli/Desktop/export.csv")

print(data)


                       Date                 Topic  \
0       03/10/2020-21:01:35         ['意大利紧急求助中国']   
1       03/10/2020-21:01:35         ['意大利紧急求助中国']   
2       03/10/2020-21:01:35         ['意大利紧急求助中国']   
3       03/10/2020-21:01:35         ['意大利紧急求助中国']   
4       03/10/2020-21:01:35         ['意大利紧急求助中国']   
...                     ...                   ...   
34144  03/31/2020, 00:00:00  ['美国监狱疫情肆虐多地在押人员抗议']   
34145  03/31/2020, 00:00:00  ['美国监狱疫情肆虐多地在押人员抗议']   
34146  03/31/2020, 00:00:00  ['美国监狱疫情肆虐多地在押人员抗议']   
34147  03/31/2020, 00:00:00  ['美国监狱疫情肆虐多地在押人员抗议']   
34148  03/31/2020, 00:00:00  ['美国监狱疫情肆虐多地在押人员抗议']   

                                                 Comment  
0      \n                    【#加拿大航空公司暂停往返意大利航班#】加拿大航...  
1      \n                    【意大利紧急求助，中方将提供口罩等医疗物资】3月...  
2      \n                    【#意大利紧急求助中国#，王毅：将向意方提供口罩...  
3      \n                    【#意大利紧急求助中国# 王毅：将向意方提供口罩...  
4      \n#印度当街烧新冠病毒怪物塑像# 是迷信吗 尊重国家的文化 但还是要做好防护才行 #意大利...  
...      